In [1]:
import os
from langchain.llms import OpenAI
from dotenv import load_dotenv
import docx
import fitz
from flask import Flask, request, render_template, redirect, url_for

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
llm = OpenAI(temperature=0.7, openai_api_key=api_key)

/tmp/ipykernel_102239/3596151289.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7, openai_api_key=api_key)


In [5]:
from langchain.prompts import PromptTemplate

In [6]:
summary_template = """ 
Summarize the text below briefly:


{text}



"""

In [7]:
prompt = PromptTemplate(
    input_variables=["text"], 
    template=summary_template
)

In [8]:
def summarize_text(text):
    '''
    takes the input string and summarizes it using llm
    '''

    input_prompt = prompt.format(text=text)

    response = llm(input_prompt)

    return response

In [9]:
summarize_text("When interacting with the API some actions such as starting a Run and adding files to vector stores are asynchronous and take time to complete. The SDK includes helper functions which will poll the status until it reaches a terminal state and then return the resulting object. If an API method results in an action that could benefit from polling there will be a corresponding version of the method ending in '_and_poll'.")

/tmp/ipykernel_102239/2390092969.py:8: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(input_prompt)


"\nThe API interactions are asynchronous and some actions may take time to complete. The SDK provides helper functions that will poll the status until it reaches a terminal state and return the resulting object. There are also similar versions of API methods that end in '_and_poll' for actions that could benefit from polling."

In [10]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
    return text

In [11]:
application = Flask(__name__)

In [12]:
@application.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        uploaded_file = request.files.get("file")
        if uploaded_file:
            file_type = uploaded_file.filename.split('.')[-1].lower()

            # Save the uploaded file to a temporary path
            file_path = os.path.join("temp", uploaded_file.filename)
            uploaded_file.save(file_path)

            # Extract text based on file type
            text = ""
            if file_type == "pdf":
                text = extract_text_from_pdf(file_path)
            elif file_type == "docx":
                text = extract_text_from_docx(file_path)
            elif file_type == "txt":
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read()

            # Limit text size for API request
            if len(text) > 1000:
                text = text[:1000]

            # Get the summary
            summary = summarize(text)

            # Remove the temporary file after processing
            os.remove(file_path)

            # Render the summary result
            return render_template("index.html", summary=summary)

    return render_template("index.html", summary=None)

# Run the Flask server
if __name__ == "__main__":
    if not os.path.exists("temp"):
        os.makedirs("temp")
    application.run(debug=True,port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/zain/summary_project/summary/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zain/summary_project/summary/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/zain/summary_project/summary/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zain/summary_project/summary/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/zain/summary_project/summary/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 331, in init_socket

SystemExit: 1

/home/zain/summary_project/summary/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
